# Generalizing a Taylor Recurrence

In [1]:
from sumpy.recurrence import _make_sympy_vec

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_off_axis_expression, get_reindexed_and_center_origin_off_axis_recurrence, _extract_idx_terms_from_recurrence

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [2]:
var = _make_sympy_vec("x", 2)
s = sp.Function("s")
n = sp.symbols("n")
i = sp.symbols("i")

In [3]:
from sumpy.expansion.diff_op import laplacian,make_identity_diff_op
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
helmholtz2d = laplacian(w) + w

In [4]:
get_reindexed_and_center_origin_off_axis_recurrence(laplace2d)[2].subs(n, 3)

2*s(1)/x1**2

In [9]:
exp = get_off_axis_expression(helmholtz2d)
exp

(x0**3*(((-6*n + (n + 2)**2 - 4)*s(n - 3)/x1**2 + (-5*n + (n + 2)**2 - 4)*s(n - 1)/x1**2)*(-5*n + (n + 4)**2 - 14)/(6*x1**2) + (-6*n + (n + 4)**2 - 16)*s(n - 1)/(6*x1**2)) + x0**2*((-6*n + (n + 3)**2 - 10)*s(n - 2)/(2*x1**2) + (-5*n + (n + 3)**2 - 9)*s(n)/(2*x1**2)) + x0*((-6*n + (n + 2)**2 - 4)*s(n - 3)/x1**2 + (-5*n + (n + 2)**2 - 4)*s(n - 1)/x1**2) + (-6*n + (n + 1)**2 + 2)*s(n - 4)/x1**2 + (-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2,
 4)

In [11]:
exp[0].simplify()

(x0**3*(-x1**2*(6*n - (n + 4)**2 + 16)*s(n - 1) + ((5*n - (n + 2)**2 + 4)*s(n - 1) + (6*n - (n + 2)**2 + 4)*s(n - 3))*(5*n - (n + 4)**2 + 14)) + 3*x1**2*(-x0**2*((5*n - (n + 3)**2 + 9)*s(n) + (6*n - (n + 3)**2 + 10)*s(n - 2)) - 2*x0*((5*n - (n + 2)**2 + 4)*s(n - 1) + (6*n - (n + 2)**2 + 4)*s(n - 3)) + 2*(-6*n + (n + 1)**2 + 2)*s(n - 4) + 2*(-5*n + (n + 1)**2 + 1)*s(n - 2)))/(6*x1**4)